## Set covering
##### Gabriele Iurlaro - Salvatore Adalberto Esposito

In [132]:
import random

from typing import Callable
from copy import deepcopy
from gx_utils import * 

In [133]:
def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

In [134]:
class State:
    def __init__(self, data: list):
        self._data = data.copy()

    def __hash__(self):
        return hash(bytes([element % 256 for sublist in self._data for element in sublist]))

    def __eq__(self, other):
        return bytes([element % 256 for sublist in self._data for element in sublist]) == bytes([element % 256 for sublist in other._data for element in sublist])

    def __lt__(self, other):
        return bytes([element % 256 for sublist in self._data for element in sublist]) < bytes([element % 256 for sublist in other._data for element in sublist])

    def __str__(self):
        return str(self._data)

    def __repr__(self):
        return repr(self._data)

    def add_action(self, action):
        self._data.append(action)
    
    def cost(self):
        return sum([len(_) for _ in self._data])
        
    @property
    def data(self):
        return self._data

    def copy_data(self):
        return self._data.copy()

In [135]:
def result(state, action):
    # Create a new state by taking the action
    new_list = deepcopy(state._data)
    new_list.append(action)
    return State(new_list)

In [136]:
def goal_test(solution, N):
    return len(set([element for sublist in solution._data for element in sublist])) == N


In [137]:
from types import new_class


def h(s, N):
    return N - len(set([element for sublist in s._data for element in sublist]))

def h2(s, N):
    return h(s, N) / s.cost()

def possible_actions(state, all_states, max = None):
    res = deepcopy(all_states)
    for list in state._data:
        # print(f"Trying to remove {list} from {res}")
        res.remove(list)
    # print(f"Returned {len(res)} actions")
    if max is None:
        return res
    else: 
        max = min(max, len(res))
        # print(f"Max: {max}")
        return [random.choice(res) for _ in range(max)]

def solution(N):
    all_states = [list(x) for x in list(set([tuple(x) for x in problem(N, seed = 42)]))]
    # print(f"Starting space has {len(all_states)} lists")
    # print(all_states)

    if len(set([element for sublist in all_states for element in sublist])) != N:
        print(f"No solution with N = {N}")
        return
    frontier = PriorityQueue()
    #print(f"Before: {all_states}")

    #print(f"{solution}")
    state = State([])
    state_cost = dict()
    state_cost[state] = state.cost()
    count = 0
    state_count = 0;

    while state is not None and not goal_test(state, N):
        for action in possible_actions(state, all_states, max = 15):
            new_state = result(state, action)
            cost = len(action)
            # cost = 0
            if new_state not in state_cost and new_state not in frontier:
                state_count += 1
                # print(f"Discovered {state_count} states")
                state_cost[new_state] = state_cost[state] + cost
                frontier.push(new_state, p = state_cost[new_state] + h(new_state, N))
            elif new_state in frontier and state_cost[new_state] > state_cost[state] + cost:
                state[new_state] = state_cost[state] + cost
        if frontier:
            state = frontier.pop()
            count += 1
            # print(f"Current state: {state}")
            # print(f"Current solution length is {count + 1}")
        else: 
            state = None
    if state is None:
        print("Not able to find the solution")
    else:
        print(f"#N = {N}, A* solution: weight = {state.cost()}, bloat = {(state.cost() - N)/N * 100:.1f}, discovered nodes {len(state_cost)} ")        
        # print(f"#N = {N}, Best-first solution: weight = {state.cost()}, bloat = {(state.cost() - N)/N * 100:.1f}, discovered nodes {len(state_cost)} ")        

In [138]:
for n in [5, 10, 20, 100, 500, 1000]:
    solution(n)
    

#N = 5, A* solution: weight = 5, bloat = 0.0, discovered nodes 598 
#N = 10, A* solution: weight = 11, bloat = 10.0, discovered nodes 4450 
#N = 20, A* solution: weight = 30, bloat = 50.0, discovered nodes 33162 


KeyboardInterrupt: 